# Advanced RAG Exercise

This notebook is designed as an exercise to build a complete Retrieval-Augmented Generation (RAG) system. In this exercise, you will integrate three main components into a single pipeline:

1. **Retrieval Module** – Retrieve relevant documents based on a query.
2. **Transformation Module** – Transform the retrieved queries.
3. **Generation Module and Evaluation** – Use the transformed data to generate responses and evaluate the overall system performance.

In [1]:
import tqdm
import glob
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings  # For generating embeddings for text chunks
import faiss
import pickle
from dotenv import load_dotenv
import os
from groq import Groq
from sentence_transformers import SentenceTransformer
import random
from sentence_transformers import CrossEncoder
import numpy as np

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



## 1. Building the RAG Pipeline

Load the data and store it in a string.

In [2]:
### load the pdf from the path
glob_path = "data/*.pdf"
text = ""
for pdf_path in tqdm.tqdm(glob.glob(glob_path)):
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
         # Extract text from all pages in the PDF
        text += " ".join(page.extract_text() for page in reader.pages if page.extract_text())

text[:50]

100%|██████████| 1/1 [00:00<00:00,  6.65it/s]


'EXERCISES WEEK 8: Convolutional Neural Networks  w'

Split the data into chunks.

In [3]:
# Create a splitter: 2000 characters per chunk with an overlap of 200 characters

# Initialize a text splitter with specified chunk size and overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
# Split the extracted text into manageable chunks
chunks = splitter.split_text(text)

# Display the resulting chunks
chunks

['EXERCISES WEEK 8: Convolutional Neural Networks  with Tensorflow&Keras  \nUse these guidelines in combination with the notebook  “CIFAR10_CNN.ipynb” :  \nWarm -up:',
 'Warm -up:  \nGet familiar with the idea of convolutions on images by experimenting with the following interactive \nlinks:  \n• https://poloclub.github.io/cnn -explainer/',
 '• https://poloclub.github.io/cnn -explainer/  \n• https://setosa.io/ev/image -kernels/  \n \nExercise 1 - Code Convolutions:',
 'Exercise 1 - Code Convolutions:  \nChoose a picture and compute a convolution on it, by implementing a e.g. vertical filter .  \n \nExercise 2: CNN for image classification with CIFAR10',
 'In this exercise we will apply the same code to the CIFAR10 Dataset.  \n(https://www.cs.toronto.edu/~kriz/cifar.html ). This also contains 10 categories of pictures, but in this',
 'case we deal with more than numbers and RGB -images  (Format, 32x32px, 3 color channels)  \nThis dataset is available to be loaded directly from Tensorflo

In [4]:
print(f"Total chunks: {len(chunks)}")
print("Preview of the first chunk:", chunks[0][:200])

Total chunks: 15
Preview of the first chunk: EXERCISES WEEK 8: Convolutional Neural Networks  with Tensorflow&Keras  
Use these guidelines in combination with the notebook  “CIFAR10_CNN.ipynb” :  
Warm -up:


## Choose an embedding model
Use the SentenceTransfomer wrapper as we have done so far.
Models are found here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
or on HuggingFace.

Embed the chunks.

In [5]:
#Name chunks muss unbedingt gemäss oberem Codesnipped sein. Dort wurden die Chunks erstellt, die wir hier embedden

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
chunk_embeddings = model.encode(chunks, convert_to_numpy=True)

## 3. Build Index and save index

In [6]:
d = chunk_embeddings.shape[1]
print(d)

384


In [7]:
index = faiss.IndexFlatL2(d)
index.add(chunk_embeddings)
print("Number of embeddings in FAISS index:", index.ntotal)

Number of embeddings in FAISS index: 15


In [8]:
faiss.write_index(index, "faiss/faiss_index.index")
with open("faiss/chunks_mapping.pkl", "wb") as f:
    pickle.dump(chunks, f)

In [9]:
index_2 = faiss.read_index("faiss/faiss_index.index")
with open("faiss/chunks_mapping.pkl", "rb") as f:
    chunks_2 = pickle.load(f)
print(len(chunks_2))
print(len(chunks))

15
15


## Load Key for language Models

In [10]:
load_dotenv()
# Access the API key using the variable name defined in the .env file
google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

## 4. Build a retriever function

arguments: query, k, index, chunks, embedding model

return: retrieved texts, distances

In [11]:
def retrieve_texts(query, k, index, token_split_texts, model):
    """
    Retrieve the top k similar text chunks and their embeddings for a given query.
    """
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    retrieved_texts = [token_split_texts[i] for i in indices[0]]
    return retrieved_texts, distances

## 5. Build an answer function
Build an answer function that takes a query, k, an index and the chunks.

return: answer

In [12]:
#Die answer-Funktion ist das Herzstück einer einfachen RAG-Pipeline (Retrieval-Augmented Generation). Du kombinierst darin:
#Eine Suchanfrage (Query)
#Die besten passenden Chunks (aus deinem FAISS-Index)
#Einen Textgenerator oder einfachen Regelmechanismus, um daraus eine Antwort zu bauen

def answer_query(query, k, index,texts):
    """
    Retrieve the top k similar text chunks for the given query using the retriever,
    inject them into a prompt, and send it to the Groq LLM to obtain an answer.
    
    Parameters:
    - query (str): The user's query.
    - k (int): Number of retrieved documents to use.
    - groq_api_key (str): Your Groq API key.
    
    Returns:
    - answer (str): The answer generated by the LLM.
    """
    # Retrieve the top k documents using your retriever function.
    # This retriever uses the following definition:
    # def retrieve(query, k):
    #     query_embedding = model.encode([query], convert_to_numpy=True)
    #     distances, indices = index.search(query_embedding, k)
    #     retrieved_texts = [token_split_texts[i] for i in indices[0]]
    #     retrieved_embeddings = np.array([chunk_embeddings[i] for i in indices[0]])
    #     return retrieved_texts, retrieved_embeddings, distances
    model_name = "paraphrase-multilingual-MiniLM-L12-v2"
    model = SentenceTransformer(model_name)
    retrieved_texts, _ = retrieve_texts(query, k, index, texts, model)
    
    # Combine the retrieved documents into a single context block.
    context = "\n\n".join(retrieved_texts)
    
    # Build a prompt that instructs the LLM to answer the query based on the context.
    prompt = (
        "Answer the following question using the provided context. "
        "Explain it as if you are explaining it to a 5 year old.\n\n"
        "Context:\n" + context + "\n\n"
        "Question: " + query + "\n"
        "Answer:"
    )
    
    # Initialize the Groq client and send the prompt.
    client = Groq(api_key=groq_api_key)
    messages = [
        {
            "role": "system",
            "content": prompt
        }
    ]
    
    llm = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile"
    )
    
    # Extract and return the answer.
    answer = llm.choices[0].message.content
    return answer


#### Test your RAG

In [13]:
query = "What is the most important factor in diagnosing asthma?"
answer = answer_query(query, 5, index, chunks)
print("LLM Answer:", answer)

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

## 6. Create a Rewriter

Take a query and an api key for the model and rewrite the query. 

Rewriting a query: A Language Model is prompted to rewrite a query to better suit a task.

Other Transfomrations are implemented in a similar fashion, this is just an example!

In [ ]:
#Der Code nimmt eine Benutzereingabe (query) und schreibt sie mithilfe eines Sprachmodells (LLM) so um, dass sie besser für eine bestimmte Anwendung geeignet ist – z. B. zur Abfrage medizinischer Richtlinien.


def rewrite_query(query, groq_api_key):
    """
    Rewrite the user's query into to potentially improve retrieval.
    Parameters:
    - query (str): The original user query.
    - groq_api_key (str): Your Groq API key.
    
    Returns:
    - rewritten_query (str): The rewritten query.
    """
    client = Groq(api_key=groq_api_key)
    # Build a prompt for rewriting the query
    rewriting_prompt = (
        "Rewrite the following query into a format, such that it can be answered by looking at medical guidelines. "
        "Keep the keywords but ensure that it is close to a format, such as in medical guidelines. Just answer with the rewritten query\n\n"
        "Query: " + query
    )
    
    messages = [
        {"role": "system", "content": rewriting_prompt}
    ]
    
    # Use the same model (for example, llama) to perform query rewriting
    llm = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile",
    )
    rewritten_query = llm.choices[0].message.content.strip()
    return rewritten_query

## 7. Implement the rewriter into your answer function

In [ ]:
#Sie nimmt eine Benutzerfrage, verbessert sie mit Query Rewriting, sucht passende Textstellen (aus einem Index), und erstellt dann eine Antwort mit einem Sprachmodell (LLM).

def answer_query_with_rewriting(query, k, index, texts, groq_api_key):
    """
    Retrieve the top k similar text chunks for the given query using a retrieval method
    with query rewriting, inject them into a prompt, and send it to the Groq LLM (using llama)
    to obtain an answer.
    
    Parameters:
    - query (str): The user's query.
    - k (int): Number of retrieved documents to use.
    - index: The FAISS index.
    - texts (list): The tokenized text chunks mapping.
    - groq_api_key (str): Your Groq API key.
    
    Returns:
    - answer (str): The answer generated by the LLM.
    """
    model_name = "paraphrase-multilingual-MiniLM-L12-v2"
    model = SentenceTransformer(model_name)
    # Use the new retrieval function with query rewriting.
    rewritten_query = rewrite_query(query, groq_api_key)    
    print("Rewritten Query:", rewritten_query) ## FYI

    retrieved_texts, _ = retrieve_texts(rewritten_query, k, index, texts, model)
    
    # Combine the retrieved documents into a single context block.
    context = "\n\n".join(retrieved_texts)
    
    # Build a prompt that instructs the LLM to answer the query based on the context.
    prompt = (
        "Answer the following question using the provided context. "
        "Explain it as if you are explaining it to a 5 year old.\n\n"
        "Context:\n" + context + "\n\n"
        "Question: " + query + "\n"
        "Answer:"
    )
    
    # Initialize the Groq client and send the prompt.
    client = Groq(api_key=groq_api_key)
    messages = [
        {"role": "system", "content": prompt}
    ]
    
    llm = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile"
    )
    
    # Extract and return the answer.
    answer = llm.choices[0].message.content
    return answer

#### Test it

In [ ]:
query = "What is the most important factor in diagnosing asthma?"
answer = answer_query_with_rewriting(query, 5, index, chunks, groq_api_key)
print("LLM Answer:", answer)

## 8 .Evaluation

Select random chunks from all your chunks, and generate a question to each of these chunks

In [ ]:
#Es werden zufällig Textausschnitte ("Chunks") ausgewählt – und das Sprachmodell (LLM) wird gebeten, zu jedem dieser Ausschnitte eine passende Frage zu stellen.

import time
import httpx  # Ensure you're catching the correct timeout exception
from openai import OpenAI
def generate_questions_for_random_chunks(chunks, num_chunks=20, max_retries=3):
    """
    Randomly selects a specified number of text chunks from the provided list,
    then generates a question for each selected chunk using the Groq LLM.

    Parameters:
    - chunks (list): List of text chunks.
    - groq_api_key (str): Your Groq API key.
    - num_chunks (int): Number of chunks to select randomly (default is 20).

    Returns:
    - questions (list of tuples): Each tuple contains (chunk, generated_question).
    """
    # Randomly select the desired number of chunks.
    selected_chunks = random.sample(chunks, num_chunks)
    
    # Initialize the Groq client once
    client = OpenAI(api_key=openai_api_key)
    
    questions = []
    for chunk in tqdm.tqdm(selected_chunks):
        # Build a prompt that asks the LLM to generate a question based on the chunk.
        prompt = (
            "Based on the following text, generate an insightful question that covers its key content:\n\n"
            "Text:\n" + chunk + "\n\n"
            "Question:"
        )
        
        messages = [
            {"role": "system", "content": prompt}
        ]
        
        generated_question = None
        attempt = 0
        
        # Try calling the API with simple retry logic.
        while attempt < max_retries:
            try:
                llm_response = client.chat.completions.create(
                     model="gpt-4o-mini",
                    messages=messages
                )
                generated_question = llm_response.choices[0].message.content.strip()
                break  # Exit the loop if successful.
            except httpx.ReadTimeout:
                attempt += 1
                print(f"Timeout occurred for chunk. Retrying attempt {attempt}/{max_retries}...")
                time.sleep(2)  # Wait a bit before retrying.
        
        # If all attempts fail, use an error message as the generated question.
        if generated_question is None:
            generated_question = "Error: Failed to generate question after several retries."
        
        questions.append((chunk, generated_question))
    
    return questions

#### Test it

In [ ]:
#Dieser Teil prüft, ob das Sprachmodell tatsächlich gute Fragen zu zufällig gewählten Textstücken ("Chunks") generiert hat.

questions = generate_questions_for_random_chunks(chunks, num_chunks=5, max_retries=2)   #Funktionsaufruf, 5 zufällige Textabschnitte aus Liste chunks
for idx, (chunk, question) in enumerate(questions, start=1):     #durchläuft jedes Tupel aus questions
    print(f"Chunk {idx}:\n{chunk[:100]}...\nGenerated Question: {question}\n")     #Zeigt die Nummer Chunk 1, Chunk 2, ... (die ersten 100 chunks)

## 9.Test the questions with your built retriever

In [ ]:
#Diese Funktion nimmt die zuvor automatisch generierten Fragen, stellt sie deinem System (Retriever + LLM) und sammelt die Antworten.

def answer_generated_questions(question_tuples, k, index, texts, groq_api_key):
    """
    For each (chunk, generated_question) tuple in the provided list, use the prebuilt
    retrieval function to generate an answer for the generated question. The function
    returns a list of dictionaries containing the original chunk, the generated question,
    and the answer.
    
    Parameters:
    - question_tuples (list of tuples): Each tuple is (chunk, generated_question)
    - k (int): Number of retrieved documents to use for answering.
    - index: The FAISS index.
    - texts (list): The tokenized text chunks mapping.
    - groq_api_key (str): Your Groq API key.
    
    Returns:
    - results (list of dict): Each dict contains 'chunk', 'question', and 'answer'.
    """
    results = []
    for chunk, question in question_tuples:
        # Use your retrieval-based answer function. Here we assume the function signature is:
        # answer_query(query, k, index, texts, groq_api_key)
        answer = answer_query(question, k, index, texts) #query, k, index,texts
        results.append({
            "chunk": chunk,
            "question": question,
            "answer": answer
        })
    return results

#### Check the results

In [ ]:
#Hier wird der allerletzte Schritt des gesamten Ablaufs ausgeführt: Du siehst dir die Resultate an, die dein System produziert hat – also die Antworten auf automatisch generierte Fragen zu zufälligen Textabschnitten.

results = answer_generated_questions(questions, 5, index, chunks, groq_api_key)

for item in results:
    print("Chunk Preview:", item['chunk'][:100])
    print("Generated Question:", item['question'])
    print("Answer:", item['answer'])
    print("-----------------------------")

## Evaluate the answers

In [ ]:
#Hier wird mit Hilfe eines LLM (Sprachmodells) geprüft, wie gut die Antworten waren, die dein System gegeben hat.

import pandas as pd
def evaluate_answers_binary(results, groq_api_key, max_retries=3):
    """
    Evaluates each answer in the results list using an LLM.
    For each result (a dictionary containing 'chunk', 'question', and 'answer'),
    it sends an evaluation prompt to the Groq LLM which outputs 1 if the answer is on point,
    and 0 if it is missing the point.
    
    Parameters:
    - results (list of dict): Each dict must contain keys 'chunk', 'question', and 'answer'.
    - groq_api_key (str): Your Groq API key.
    - max_retries (int): Maximum number of retries if the API call times out.
    
    Returns:
    - df (pandas.DataFrame): A dataframe containing the original chunk, question, answer, and evaluation score.
    """
    evaluations = []
    client = OpenAI(api_key=openai_api_key)
    
    for item in tqdm.tqdm(results, desc="Evaluating Answers"):
        # Build the evaluation prompt.
        prompt = (
            "Evaluate the following answer to the given question. "
            "If the answer is accurate and complete, reply with 1. "
            "If the answer is inaccurate, incomplete, or otherwise not acceptable, reply with 0. "
            "Do not include any extra text.\n\n"
            "Question: " + item['question'] + "\n\n"
            "Answer: " + item['answer'] + "\n\n"
            "Context (original chunk): " + item['chunk'] + "\n\n"
            "Evaluation (1 for good, 0 for bad):"
        )
        
        messages = [{"role": "system", "content": prompt}]
        
        generated_eval = None
        attempt = 0
        
        # Retry logic in case of timeouts or errors.
        while attempt < max_retries:
            try:
                llm_response = client.chat.completions.create(
                    messages=messages,
                    model="4o-mini"
                )
                generated_eval = llm_response.choices[0].message.content.strip()
                break  # Exit the retry loop if successful.
            except httpx.ReadTimeout:
                attempt += 1
                print(f"Timeout occurred during evaluation. Retrying attempt {attempt}/{max_retries}...")
                time.sleep(2)
            except Exception as e:
                attempt += 1
                print(f"Error during evaluation: {e}. Retrying attempt {attempt}/{max_retries}...")
                time.sleep(2)
        
        # If no valid evaluation was produced, default to 0.
        if generated_eval is None:
            generated_eval = "0"
        
        # Convert the response to an integer (1 or 0).
        try:
            score = int(generated_eval)
            if score not in [0, 1]:
                score = 0
        except:
            score = 0
        
        evaluations.append(score)
    
    # Add the evaluation score to each result.
    for i, item in enumerate(results):
        item['evaluation'] = evaluations[i]
    
    # Create a dataframe for manual review.
    df = pd.DataFrame(results)
    return df

### Display them

In [ ]:
#darstellen

df_evaluations = evaluate_answers_binary(results, openai_api_key)
display(df_evaluations)